# ETFs with Stats

In [2]:
%%time

import pandas as pd
import sqlite3 as db
import requests
import functools
import numpy as np 
from bs4 import BeautifulSoup

###### Returns all available ETF links ######
s = []
r  = requests.get("http://www.etf.com/channels")
data = r.text
soup = BeautifulSoup(data, "lxml")
for link in soup.find_all('a'):
    if '/channels/' in str(link.get('href')):
        s.append('http://www.etf.com' + str(link.get('href')))
# print(s)

###### Returns data from all ETFs ETNs ######
# Load data from all ETF/ETN links
frames = None
for i in range(0, len(s)):  # 10 for test
    
    print("Scraping data from {}".format(s[i]))
    print("Page {} in ".format(i) + str(len(s)))

    try:
        
        # Download industry ETF data
        df = pd.read_html(requests.get(s[i],headers={'User-agent': 'Mozilla/5.0'}).text)
        del df[len(df) -1] # Deletes last cell w/out data
        
        # Fix columns with flows/changes from stats section
        for j in range(0, len(df)):
            # Fixes Columns without names
            if df[j].columns[0] == 0:  
                df[j].columns = df[j].iloc[0]
                df[j] = df[j].iloc[1:,]
                df[j].columns = [x.title() for x in df[j].columns]
                # print(df[j])
        
        # Merge and clean per industry
        df = functools.reduce(lambda x, y: pd.merge(x, y, on = list(set(x).intersection(y)), how = 'outer'), df)
        del df['% Change*']
        del df['Net Flows*']
        df = df.replace(np.nan, '', regex=True)
        df = df.replace('--', '', regex=False)
        df = df.replace('%', '', regex=True)
        df['AUM'] = df['AUM'].apply(lambda x: x.replace('$',''))
        df = df.drop_duplicates(keep=False)
        
        # Add Industry Columns
        #         df['type_link'] = s[i].split('/')[-1].replace('-etfs', '')
        #         df['link'] = s[i]
        
        # Convert AUM ('B', 'M', 'K') to integers
        from decimal import Decimal
        d = {'K': 3, 'M': 6,'B': 9}
        def text_to_num(text):
            if text == '':
                return ''
            elif text[-1] in d:
                num, magnitude = text[:-1], text[-1]
                return int(Decimal(num) * 10 ** d[magnitude])
            else:
                return int(Decimal(text))

        df['AUM'] = [text_to_num(a) for a in df['AUM']]
        frames = df.append(frames)
        
    except:
        next

try:
    # Sort by category and remove duplicate lines and lines without data
    frames = frames[frames.Leveraged != ''] # Removes duplicate rows shared across pages
    frames = frames.drop_duplicates(subset=['Ticker'], keep=False)
    frames = frames.sort_values(by=['Asset Class', 'Strategy', 'Region', 'Geography', 'Category', 'Focus'])
    frames = frames.drop([' ', ' .1'], 1) # Drop columns without data
    frames.rename(columns={'E': 'Efficiency Score', 'T': 'Tradability Score', 'F': 'Fit Neutrality Score'}, inplace=True)
    frames = frames.reset_index(drop=True) # Reindex
    frames['Download UTC Date'] = datetime.now(timezone.utc).strftime("%Y/%m/%d")
except:
    print('Error in last segment')

Scraping data from http://www.etf.com/channels/alpha-seeking-etfs
Page 0 in 1786
Scraping data from http://www.etf.com/channels/energy-etfs
Page 1 in 1786
Scraping data from http://www.etf.com/channels/oil-etfs
Page 2 in 1786
Scraping data from http://www.etf.com/channels/biotech-etfs
Page 3 in 1786
Scraping data from http://www.etf.com/channels/europe-etfs
Page 4 in 1786
Scraping data from http://www.etf.com/channels/reit-etfs
Page 5 in 1786
Scraping data from http://www.etf.com/channels/bond-etfs
Page 6 in 1786
Scraping data from http://www.etf.com/channels/fixed-income-etfs
Page 7 in 1786
Scraping data from http://www.etf.com/channels/china-etfs
Page 8 in 1786
Scraping data from http://www.etf.com/channels/gold-etfs
Page 9 in 1786
Scraping data from http://www.etf.com/channels/silver-etfs
Page 10 in 1786
Scraping data from http://www.etf.com/channels/commodity-etfs
Page 11 in 1786
Scraping data from http://www.etf.com/channels/health-care-etfs
Page 12 in 1786
Scraping data from http

In [3]:
frames

,Ticker,Fund Name,Issuer,Expense Ratio,AUM,Spread %,Segment,1 Month,3 Month,1 Year,...,MAX LT CapitalGains Rate,Max ST CapitalGains Rate,TaxReporting,MSCI ESGQuality Score,ESG ScorePeer Rank,ESG ScoreGlobal Rank,Carbon IntensityTons CO2e/$M Sales,SustainableImpact Exposure,SRI ScreeningCriteria Exposure,Download UTC Date
0,DALT,Anfield Capital Diversified Alternatives ETF,Northern Lights,1.30,29320000,0.61,Alternatives: Absolute Returns,-0.36,3.56,,...,20,39.6,1099,,,,,,,2018/07/05
1,VEGA,AdvisorShares STAR Global Buy-Write ETF,AdvisorShares,2.05,17060000,0.56,Alternatives: Absolute Returns,-0.22,4.81,6.75,...,20,39.6,1099,,,,,,,2018/07/05
2,LALT,Invesco Multi-Strategy Alternative ETF,Invesco,0.98,4430000,1.07,Alternatives: Absolute Returns,0.92,0.23,1.13,...,20,39.6,1099,,,,,,,2018/07/05
3,FMF,First Trust Morningstar Managed Futures Strate...,First Trust,0.95,12170000,0.39,Alternatives: Absolute Returns,1.76,3.04,6.77,...,20,39.6,1099,,,,,,,2018/07/05
4,VMAX,REX VolMAXX Long VIX Futures Strategy ETF,Exchange Traded Concepts,2.85,2810000,0.56,Alternatives: U.S. - Volatility,3.09,-16.45,-23.96,...,20,39.6,1099,,,,,,,2018/07/05
5,RALS,ProShares RAFI Long/Short ETF,ProShares,0.95,11840000,0.50,Alternatives: Absolute Returns,-0.33,-1.19,-2.40,...,20,39.6,1099,4.92 / 10,42.25,27.09,328.29,3.92,11.54,2018/07/05
6,DYB,WisdomTree Dynamic Bearish U.S. Equity Fund,WisdomTree,0.48,9090000,0.22,Alternatives: Absolute Returns,-0.43,-0.15,1.17,...,20,39.6,1099,4.71 / 10,9.62,21.37,239.42,4.56,2.32,2018/07/05
7,MRGR,ProShares Merger ETF,ProShares,0.75,4470000,0.29,Alternatives: Absolute Returns,-0.47,0.60,-1.65,...,20,39.6,1099,,,,,,,2018/07/05
8,QED,IQ Hedge Event-Driven Tracker ETF,IndexIQ,0.99,4240000,0.19,Alternatives: Absolute Returns,0.33,2.50,4.95,...,20,39.6,1099,,,,,,,2018/07/05
9,QXTR,QuantX Risk Managed Multi-Asset Total Return ETF,Northern Lights,1.45,36710000,0.33,Alternatives: Absolute Returns,0.13,5.54,6.67,...,20,39.6,1099,,,,,,,2018/07/05


In [4]:
# frames.to_csv("etf.csv", sep=',', encoding='utf-8')

In [ ]:
# For writing to SQLite see:
# http://www.pythonforfinance.net/2016/04/11/sqlite-database-python-web-scrape/#comment-304